In [0]:
import sys
workspace = dbutils.widgets.get('workspace')
sys.path.insert(0, f"{workspace}scr/")

from libs.rename_columns import *
from libs.create_bronze_table import create_table
from datetime import datetime
from pyspark.sql.functions import lit

In [0]:
path_source = dbutils.widgets.get('path_source')
catalog = dbutils.widgets.get('catalog')
schema  = dbutils.widgets.get('schema')
tables  = ['cnaes', 'empresas', 'estabelecimentos', 'socios', 'motivos', 'municipios', 'naturezas', 'paises', 'qualificacoes', 'simples']

In [0]:
list_folders = dbutils.fs.ls(path_source)

if len(list_folders) < 2:
    print('Arquivos não existentes')
else:
    for table in tables:
        # Criando tabela se ela não existir
        create_table(
            spark=spark, 
            catalog=catalog, 
            schema=schema, 
            table=table
        )

        # realizando a leitura dos arquivos e criando um df spark
        df = (spark.read
            .option('delimiter', ';')
            .option('header', False)
            .option('inferSchema', False)
            .option('encoding', 'latin1')
            .csv(f"{path_source}/{table}*")
        )

        table_columns = eval(table)

        # Renomear as colunas
        for old, new in table_columns.items():
            df = df.withColumnRenamed(old, new)

        # Adicionando uma coluna com o nome da tabela
        df = df.withColumn('data_ingestao', lit(datetime.today().date()))

        # Escrevendo em formato delta
        (df.write
            .mode('overwrite')
            .format('delta')
            .option("mergeSchema", "true")
            .saveAsTable(f"{catalog}.{schema}.{table}")
        )

        print(f"{table} populada com sucesso")

    # Limpando os arquivos da pasta
    for file in list_folders:
        if file.name != 'temp/':
            dbutils.fs.rm(f"{path_source}/{file.name}", True)
